Workspace
=========

The core element of a project is the ``Workspace``. A project Workspace holds core information about the author, version and all entities stored in the `geoh5` file. It also knows how to create the core structure needed by [Geoscience Analyst](https://mirageoscience.com/mining-industry-software/geoscience-analyst/) for visualization.

![workspace](./images/workspace.png)


You can either open an existing project or create a new project by simply entering the desired file name.

In [ ]:
from geoh5py.workspace import Workspace

# Create a new project
workspace = Workspace("my_project.geoh5")

Et voila!
![blankworkspace](./images/blankworkspace.png){width="50%"}